## Import

In [48]:
# !pip3 install albumentations ;!pip3 install opencv-python ; !pip3 install tqdm

In [1]:
import wandb
wandb.login()

Failed to detect the name of this notebook, you can set it manually with the WANDB_NOTEBOOK_NAME environment variable to enable code saving.
wandb: Currently logged in as: anthoniy. Use `wandb login --relogin` to force relogin


True

In [ ]:
import random

# start a new wandb run to track this script
wandb.init(
    # set the wandb project where this run will be logged
    project="Dacon-papering-competition",
    
    # track hyperparameters and run metadata
    config={
    'LEARNING_RATE':3e-4,
    'IMG_SIZE':224,
    "architecture": "Efficientnet-b0",
    'EPOCHS':30,
    'SEED':42,
    'BATCH_SIZE':64,
    "dataset": "Dacon-papering-competition",
    }# hyperparameter
)


In [ ]:
# hyperparameter
CFG = wandb.config
CFG

{'LEARNING_RATE': 0.003, 'IMG_SIZE': 224, 'architecture': 'Efficientnet-b0', 'EPOCHS': 50, 'SEED': 42, 'BATCH_SIZE': 32, 'dataset': 'Dacon-papering-competition'}

In [ ]:
import random
import pandas as pd
import numpy as np
import os
import re
import glob
import cv2

import torch
import torch.nn as nn
import torch.optim as optim
import torch.nn.functional as F
from torch.utils.data import Dataset, DataLoader, WeightedRandomSampler

import albumentations as A
from albumentations.pytorch.transforms import ToTensorV2
import torchvision.models as models

from sklearn.model_selection import train_test_split
from sklearn import preprocessing
from sklearn.metrics import f1_score
from sklearn.metrics import classification_report
from tqdm.auto import tqdm

# torch.multiprocessing import
from torch import multiprocessing
from collections import OrderedDict, Counter
from sklearn.utils import resample
import warnings
warnings.filterwarnings(action='ignore') 

In [ ]:
# 경로지정
import os
os.chdir('../DATA')
os.getcwd()

'c:\\Users\\DoSungjin\\Documents\\GitHub\\Dacon_papering_classification\\DATA'

In [ ]:
import torch
# Set the device to CPU or GPU depending on availability
device = torch.device('cuda:0') if torch.cuda.is_available() else torch.device('cpu')
torch.cuda.is_available()

True

## Fixed RandomSeed

In [ ]:
def seed_everything(seed):
    random.seed(seed)
    os.environ['PYTHONHASHSEED'] = str(seed)
    np.random.seed(seed)
    torch.manual_seed(seed)
    torch.cuda.manual_seed(seed)
    torch.backends.cudnn.deterministic = True
    torch.backends.cudnn.benchmark = True

seed_everything(CFG['SEED']) # Seed 고정

## Data Pre-processing

In [ ]:
os.getcwd()

'c:\\Users\\DoSungjin\\Documents\\GitHub\\Dacon_papering_classification\\DATA'

In [ ]:
import os

# 데이터셋 디렉토리 경로
dataset_dir = "new_train"

# 모든 이미지 파일 경로 리스트
all_img_list = []
folder_list = []
train_file_list = os.listdir(dataset_dir)
for item in train_file_list:
    item_path = os.path.join(dataset_dir, item)
    for file in os.listdir(item_path):
        all_img_list.append(os.path.join(item_path, file))
        folder_list.append(item)

In [ ]:
# # folder name list
ori_names = os.listdir('ori_train')

new_names = ['furniture_repair', 'cleaning_mop_holder_repair', 'mold', 'twist', 'rust_contamination', 'wobbling', 'fabric_defect', 'molding_repair', 'stain', 'plaster_repair', 'pollution', 'typographical_error', 'crying', 'defective_joint', 'windowdoor_frame_repair', 'cracking', 'excessive_gap', 'piece', 'damage']

# print(len(ori_names), len(new_names))
# os.chdir('new_train')
# # # folder rename
# for ori_name, new_name in zip(ori_names, new_names):
#     os.rename(ori_name, new_name)

# print(os.listdir())
# os.chdir('../')

In [ ]:
for a,b in zip(ori_names, new_names):
    len_a = len(os.listdir(f'ori_train/{a}'))
    len_b = len(os.listdir(f'new_train/{b}'))
    if len_a == len_b:
        print('go on')
    else: 
        print('stop')

go on
go on
go on
go on
go on
go on
go on
go on
go on
go on
go on
go on
go on
go on
go on
go on
go on
go on
go on


In [1]:
import torch
import gc

# If you are using CUDA
torch.cuda.empty_cache()
gc.collect()

c:\Users\DoSungjin\anaconda3\envs\gpu-env\lib\site-packages\tqdm\auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


0

In [ ]:
df = pd.DataFrame(columns=['img_path', 'label'])
df['img_path'] = all_img_list
df['label'] = df['img_path'].apply(lambda x : str(x).split('\\')[1])

In [ ]:
Counter(df['label'])

Counter({'cleaning_mop_holder_repair': 307,
         'cracking': 162,
         'crying': 22,
         'damage': 1405,
         'defective_joint': 17,
         'excessive_gap': 5,
         'fabric_defect': 99,
         'furniture_repair': 12,
         'mold': 145,
         'molding_repair': 130,
         'piece': 51,
         'plaster_repair': 57,
         'pollution': 595,
         'rust_contamination': 14,
         'stain': 3,
         'twist': 210,
         'typographical_error': 142,
         'windowdoor_frame_repair': 27,
         'wobbling': 54})

In [ ]:
group_100_or_less = []
group_100_to_600 = []
group_600_or_more = []

for key, value in Counter(df['label']).items():
    if value <= 100:
        group_100_or_less.append(key)
    elif value <= 600:
        group_100_to_600.append(key)
    else:
        group_600_or_more.append(key)

# 결과 출력
print("100개 이하 그룹:", group_100_or_less)
print("100~600개 그룹:", group_100_to_600)
print("600개 이상 그룹:", group_600_or_more)


100개 이하 그룹: ['crying', 'defective_joint', 'excessive_gap', 'fabric_defect', 'furniture_repair', 'piece', 'plaster_repair', 'rust_contamination', 'stain', 'windowdoor_frame_repair', 'wobbling']
100~600개 그룹: ['cleaning_mop_holder_repair', 'cracking', 'mold', 'molding_repair', 'pollution', 'twist', 'typographical_error']
600개 이상 그룹: ['damage']


In [ ]:
from sklearn.utils import resample

# 100개 이하 그룹 오버샘플링
minor_class_samples_100_or_less = df[df['label'].isin(group_100_or_less)]
minor_class_count_100_or_less = len(minor_class_samples_100_or_less)
oversample_count_100_or_less = int(minor_class_count_100_or_less * 5)
oversampled_minor_class_100_or_less = resample(minor_class_samples_100_or_less, replace=True, n_samples=oversample_count_100_or_less, random_state=42)

# 100~600개 그룹 오버샘플링
minor_class_samples_100_to_600 = df[df['label'].isin(group_100_to_600)]
minor_class_count_100_to_600 = len(minor_class_samples_100_to_600)
oversample_count_100_to_600 = int(minor_class_count_100_to_600 * 1.5)
oversampled_minor_class_100_to_600 = resample(minor_class_samples_100_to_600, replace=True, n_samples=oversample_count_100_to_600, random_state=42)

# 600개 이상 그룹 오버샘플링
minor_class_samples_600_or_more = df[df['label'].isin(group_600_or_more)]
oversampled_minor_class_600_or_more = minor_class_samples_600_or_more  # 그대로 유지

# 오버샘플링된 소수 클래스 데이터셋과 다수 클래스 데이터셋 합치기
df_oversampled = pd.concat([oversampled_minor_class_100_or_less, oversampled_minor_class_100_to_600, oversampled_minor_class_600_or_more])

# 샘플 순서 랜덤하게 섞기
df_oversampled = df_oversampled.sample(frac=1, random_state=42).reset_index(drop=True)


In [ ]:
from sklearn.model_selection import train_test_split

train, val, _, _ = train_test_split(df_oversampled, df_oversampled['label'], test_size=0.2, stratify=df_oversampled['label'], random_state=CFG['SEED'])


## Label-Encoding

In [ ]:
le = preprocessing.LabelEncoder()
train['label'] = le.fit_transform(train['label'])
val['label'] = le.transform(val['label'])
train

,img_path,label
3584,new_train\pollution\388.png,12
4431,new_train\damage\842.png,3
2227,new_train\mold\9.png,8
1164,new_train\crying\2.png,2
1171,new_train\fabric_defect\72.png,6
...,...,...
2720,new_train\piece\33.png,10
5319,new_train\damage\520.png,3
5280,new_train\typographical_error\99.png,16
3444,new_train\wobbling\17.png,18


## CustomDataset

In [ ]:
# class CustomDataset(Dataset):
#     def __init__(self, img_path_list, label_list, transforms=None):
#         self.img_path_list = img_path_list
#         self.label_list = label_list
#         self.transforms = transforms
        
#     def __getitem__(self, index):
#         img_path = self.img_path_list[index]
        
#         image = cv2.imread(img_path)
        
#         if self.transforms is not None:
#             image = self.transforms(image=image)['image']
        
#         if self.label_list is not None:
#             label = self.label_list[index]
#             return image, label
#         else:
#             return image
        
#     def __len__(self):
#         return len(self.img_path_list)

In [ ]:
# import albumentations as A
# from albumentations.pytorch.transforms import ToTensorV2

# class_aware_augmentation_minor = A.Compose([
#     A.Resize(CFG['IMG_SIZE'], CFG['IMG_SIZE']),
#     A.HorizontalFlip(p=0.5),
#     A.RandomBrightnessContrast(p=0.2),
#     A.OneOf([
#         A.Rotate(limit=10),
#         A.IAAAffine(rotate=10)
#     ], p=0.5),
#     A.Normalize(mean=(0.485, 0.456, 0.406), std=(0.229, 0.224, 0.225)),
#     ToTensorV2(),
# ])

# general_augmentation_major = A.Compose([
#     A.Resize(CFG['IMG_SIZE'], CFG['IMG_SIZE']),
#     A.HorizontalFlip(p=0.5),
#     A.Rotate(limit=10, p=0.5),
#     A.RandomBrightnessContrast(p=0.2),
#     A.Normalize(mean=(0.485, 0.456, 0.406), std=(0.229, 0.224, 0.225)),
#     ToTensorV2(),
# ])

# def apply_augmentation(image, label):
#     if label in keys_under_100:
#         augmented = class_aware_augmentation_minor(image=image)
#     else:
#         augmented = general_augmentation_major(image=image)
#     return augmented['image'], label

# test_transform = A.Compose([
#     A.Resize(CFG['IMG_SIZE'], CFG['IMG_SIZE']),
#     A.Normalize(mean=(0.485, 0.456, 0.406), std=(0.229, 0.224, 0.225)),
#     ToTensorV2()
# ])


In [ ]:
train

,img_path,label
3584,new_train\pollution\388.png,12
4431,new_train\damage\842.png,3
2227,new_train\mold\9.png,8
1164,new_train\crying\2.png,2
1171,new_train\fabric_defect\72.png,6
...,...,...
2720,new_train\piece\33.png,10
5319,new_train\damage\520.png,3
5280,new_train\typographical_error\99.png,16
3444,new_train\wobbling\17.png,18


In [ ]:
os.getcwd()

'c:\\Users\\DoSungjin\\Documents\\GitHub\\Dacon_papering_classification\\DATA'

In [ ]:
train['img_path'].values

array(['new_train\\pollution\\388.png', 'new_train\\damage\\842.png',
       'new_train\\mold\\9.png', ...,
       'new_train\\typographical_error\\99.png',
       'new_train\\wobbling\\17.png', 'new_train\\mold\\73.png'],
      dtype=object)

In [ ]:
import cv2
import albumentations as A
from albumentations.pytorch.transforms import ToTensorV2
from torch.utils.data import DataLoader

# class ClassAwareAugmentationMinor:
#     def __init__(self, img_size):
#         self.transform = A.Compose([
#             A.Resize(img_size, img_size),
#             A.HorizontalFlip(p=0.5),
#             A.OneOf([
#                 A.Rotate(limit=10),
#                 A.IAAAffine(rotate=10)
#             ], p=0.5),
#             A.Normalize(mean=(0.485, 0.456, 0.406), std=(0.229, 0.224, 0.225)),
#             ToTensorV2(),
#         ])

#     def __call__(self, image):
#         augmented = self.transform(image=image)
#         return augmented['image']


import albumentations as A
from albumentations.pytorch.transforms import ToTensorV2



train_transform = A.Compose([
    A.Resize(CFG['IMG_SIZE'], CFG['IMG_SIZE']),
    A.HorizontalFlip(p=0.5),
    A.Rotate(limit=10, p=0.5),
    A.RandomBrightnessContrast(p=0.2),
    ToTensorV2() 
])



test_transform = A.Compose([
    A.Resize(CFG['IMG_SIZE'], CFG['IMG_SIZE']),
    A.Normalize(mean=(0.485, 0.456, 0.406), std=(0.229, 0.224, 0.225)),
    ToTensorV2()
])


class CustomDataset:
    def __init__(self, img_paths, labels, train_transform): #, transform_minor):
        self.img_paths = img_paths
        self.labels = labels
        self.train_transform = train_transform
    def __len__(self):
        return len(self.img_paths)

    def __getitem__(self, idx):
        img_path = self.img_paths[idx]
        label = self.labels[idx]
        image = cv2.imread(img_path)
        image = cv2.cvtColor(image, cv2.COLOR_BGR2RGB)
        image = train_transform(image=image)['image']
        
        return image, label


train_dataset = CustomDataset(train['img_path'].values, train['label'].values, train_transform)
train_loader = DataLoader(train_dataset, batch_size = CFG['BATCH_SIZE'], shuffle=True, num_workers=0)

val_dataset = CustomDataset(val['img_path'].values, val['label'].values, test_transform)
val_loader = DataLoader(val_dataset, batch_size=CFG['BATCH_SIZE'], shuffle=False, num_workers=0)

## Model Define

In [ ]:
class BaseModel(nn.Module):
    def __init__(self, num_classes=len(le.classes_)):
        super(BaseModel, self).__init__()
        self.backbone = models.efficientnet_b0(pretrained=True)
        self.classifier = nn.Linear(1000, num_classes)
        
    def forward(self, x):
        x = self.backbone(x)
        x = self.classifier(x)
        return x

In [ ]:
next(iter(train_loader))[1][0]

tensor(15, dtype=torch.int32)

## Train

In [253]:
def train(model, optimizer, train_loader, val_loader, scheduler, device):
    model.to(device)
    criterion = nn.CrossEntropyLoss().to(device)
    
    best_score = 0
    best_model = None
    
    for epoch in range(1, CFG['EPOCHS']+1):
        model.train()
        train_loss = []
        for imgs, labels in tqdm(iter(train_loader)):
            imgs = imgs.float().to(device)
            label_map = {label: i for i, label in enumerate(set(labels.tolist()))}
            numerical_labels = [label_map[int(label.item())] for label in labels]
            labels = torch.tensor(numerical_labels, dtype=torch.long)
            labels = labels.to(device)
            optimizer.zero_grad()
            
            output = model(imgs)
            loss = criterion(output, labels)
            
            loss.backward()
            optimizer.step()
            
            train_loss.append(loss.item())
                    
        _val_loss, _val_score = validation(model, criterion, val_loader, device)
        _train_loss = np.mean(train_loss)
        print(f'Epoch [{epoch}], Train Loss : [{_train_loss:.5f}] Val Loss : [{_val_loss:.5f}] Val Weighted F1 Score : [{_val_score:.5f}]')
        # wandb에 로그 기록
        wandb.log({'epoch': epoch, 'train_loss': _train_loss, 'val_loss': _val_loss, 'val_score': _val_score})
        
        print(f'Epoch [{epoch}], Train Loss : [{_train_loss:.5f}] Val Loss : [{_val_loss:.5f}] Val Weighted F1 Score : [{_val_score:.5f}]')
    
        if scheduler is not None:
            scheduler.step(_val_score)
            
        if best_score < _val_score:
            best_score = _val_score
            best_model = model

        # wandb에 최적의 모델 저장
    wandb.save('best_model.pth')

    return best_model

In [254]:
def validation(model, criterion, val_loader, device):
    model.eval()
    val_loss = []
    preds, true_labels = [], []

    with torch.no_grad():
        for imgs, labels in tqdm(iter(val_loader)):
            imgs = imgs.float().to(device)
            label_map = {label: i for i, label in enumerate(set(labels.tolist()))}
            numerical_labels = [label_map[int(label.item())] for label in labels]
            labels = torch.tensor(numerical_labels, dtype=torch.long)
            labels = labels.to(device)


            pred = model(imgs)
            
            loss = criterion(pred, labels)
            
            preds += pred.argmax(1).detach().cpu().numpy().tolist()
            true_labels += labels.detach().cpu().numpy().tolist()
            
            val_loss.append(loss.item())
        
        _val_loss = np.mean(val_loss)
        _val_score = f1_score(true_labels, preds, average='weighted')
    
    return _val_loss, _val_score

## Run!!

In [252]:
model = BaseModel()
model.eval()
optimizer = torch.optim.AdamW(params = model.parameters(), lr = CFG["LEARNING_RATE"])
scheduler = torch.optim.lr_scheduler.ReduceLROnPlateau(optimizer, mode='max', factor=0.5, patience=2, threshold_mode='abs', min_lr=1e-8, verbose=True)

infer_model = train(model, optimizer, train_loader, val_loader, scheduler, device)

100%|██████████| 36/36 [00:10<00:00,  3.56it/s]


Epoch [1], Train Loss : [2.62615] Val Loss : [2.50278] Val Weighted F1 Score : [0.12427]


100%|██████████| 36/36 [00:10<00:00,  3.58it/s]


Epoch [2], Train Loss : [2.25091] Val Loss : [2.18359] Val Weighted F1 Score : [0.20064]


100%|██████████| 36/36 [00:10<00:00,  3.59it/s]


...

Epoch [35], Train Loss : [1.04032] Val Loss : [1.78351] Val Weighted F1 Score : [0.36217]
Epoch 00035: reducing learning rate of group 0 to 2.3438e-05.


100%|██████████| 36/36 [00:10<00:00,  3.38it/s]


Epoch [36], Train Loss : [1.11329] Val Loss : [1.83533] Val Weighted F1 Score : [0.37339]


 31%|███       | 11/36 [00:02<00:06,  3.73it/s]


KeyboardInterrupt: 

## Inference

In [ ]:
test = pd.read_csv('test.csv')
test

,id,img_path
0,TEST_000,./test/000.png
1,TEST_001,./test/001.png
2,TEST_002,./test/002.png
3,TEST_003,./test/003.png
4,TEST_004,./test/004.png
...,...,...
787,TEST_787,./test/787.png
788,TEST_788,./test/788.png
789,TEST_789,./test/789.png
790,TEST_790,./test/790.png


In [255]:
import torch
import gc

# If you are using CUDA
torch.cuda.empty_cache()
gc.collect()

8459

In [258]:
!nvidia-smi

Fri May 12 12:30:44 2023       
+---------------------------------------------------------------------------------------+
| NVIDIA-SMI 531.61                 Driver Version: 531.61       CUDA Version: 12.1     |
|-----------------------------------------+----------------------+----------------------+
| GPU  Name                      TCC/WDDM | Bus-Id        Disp.A | Volatile Uncorr. ECC |
| Fan  Temp  Perf            Pwr:Usage/Cap|         Memory-Usage | GPU-Util  Compute M. |
|                                         |                      |               MIG M. |
|=========================================+======================+======================|
|   0  NVIDIA GeForce RTX 3070 L...  WDDM | 00000000:01:00.0 Off |                  N/A |
| N/A   40C    P8               10W /  N/A|   5153MiB /  8192MiB |      0%      Default |
|                                         |                      |                  N/A |
+-----------------------------------------+----------------------+--

In [ ]:
test_dataset = CustomDataset(test['img_path'].values, None, test_transform)
test_loader = DataLoader(test_dataset, batch_size=CFG['BATCH_SIZE'], shuffle=False, num_workers=0)

In [60]:
def inference(model, test_loader, device):
    model.eval()
    preds = []
    with torch.no_grad():
        for imgs in tqdm(iter(test_loader)):
            imgs = imgs.float().to(device)
            
            pred = model(imgs)
            
            preds += pred.argmax(1).detach().cpu().numpy().tolist()
    
    preds = le.inverse_transform(preds)
    return preds

In [61]:
preds = inference(infer_model, test_loader, device)

100%|██████████| 25/25 [00:13<00:00,  1.81it/s]


## Submission

In [62]:
submit = pd.read_csv('sample_submission.csv')

In [63]:
submit['label'] = preds

In [64]:
submit

,id,label
0,TEST_000,damage
1,TEST_001,damage
2,TEST_002,damage
3,TEST_003,molding_repair
4,TEST_004,pollution
...,...,...
787,TEST_787,pollution
788,TEST_788,cracking
789,TEST_789,pollution
790,TEST_790,pollution


In [65]:
# result folder 생성
from datetime import datetime, timezone, timedelta

kst = timezone(timedelta(hours=9))
train_serial =  datetime.now(kst).strftime('%Y%m%d_%H%M%S')

Record_path = os.path.join('result', train_serial)

os.makedirs(Record_path, exist_ok=True)




In [66]:
# Dictionary to map old column names to new column names
name_map = dict(zip(new_names, ori_names))
name_map


# Rename columns using the rename() method
submit_test = submit.replace(name_map)
submit_test

,id,label
0,TEST_000,훼손
1,TEST_001,훼손
2,TEST_002,훼손
3,TEST_003,몰딩수정
4,TEST_004,오염
...,...,...
787,TEST_787,오염
788,TEST_788,터짐
789,TEST_789,오염
790,TEST_790,오염


In [67]:
submit_test.to_csv(os.path.join(Record_path,'submission.csv'), index=False)

In [76]:
from collections import Counter
Counter(submit_test['label'])

Counter({'훼손': 375,
         '몰딩수정': 27,
         '오염': 187,
         '오타공': 36,
         '터짐': 26,
         '걸레받이수정': 18,
         '가구수정': 11,
         '곰팡이': 23,
         '녹오염': 9,
         '석고수정': 11,
         '피스': 15,
         '꼬임': 37,
         '면불량': 9,
         '창틀,문틀수정': 4,
         '틈새과다': 1,
         '이음부불량': 2,
         '울음': 1})

In [73]:
dict(CFG)

{'LEARNING_RATE': 0.0003,
 'IMG_SIZE': 224,
 'architecture': 'Efficientnet-b0',
 'EPOCHS': 10,
 'SEED': 42,
 'BATCH_SIZE': 32,
 'dataset': 'Dacon-papering-competition'}

In [74]:
# model information 저장
import json
model_info = dict(CFG)
with open(os.path.join(Record_path,'model_info.json'), 'w') as f:
    json.dump(model_info, f)